In [114]:
import sys
import statsmodels.api as sm
from sklearn.metrics import root_mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
import pandas as pd
import numpy as np
sys.path.append('./')
import problem

# Import des données

In [80]:
X_train, y_train = problem.get_train_data(True)
X_test, y_test = problem.get_test_data(True)

assert X_train.shape[1] == 284 + 331695

# Récupération des ROI

In [228]:
ROI_train = pd.read_csv("../data/train_rois.csv")
ROI_test = pd.read_csv("../data/test_rois.csv")

In [229]:
ROI_train

,participant_id,CSF_Vol,GM_Vol,WM_Vol,l3thVen_GM_Vol,r3thVen_GM_Vol,l4thVen_GM_Vol,r4thVen_GM_Vol,lAcc_GM_Vol,rAcc_GM_Vol,...,lSupParLo_CSF_Vol,rSupParLo_CSF_Vol,lSupTemGy_CSF_Vol,rSupTemGy_CSF_Vol,lTemPo_CSF_Vol,rTemPo_CSF_Vol,lInfFroAngGy_CSF_Vol,rInfFroAngGy_CSF_Vol,lTemTraGy_CSF_Vol,rTemTraGy_CSF_Vol
0,651,334.031002,603.083565,560.465707,0.046239,0.052240,0.063827,0.068097,0.419533,0.427702,...,9.417022,8.434683,2.347495,2.171560,5.581092,5.706735,1.890379,1.612733,0.282384,0.266220
1,431,300.991328,603.101354,594.425495,0.052599,0.047901,0.088186,0.105241,0.373451,0.377817,...,7.536400,9.255206,1.916260,2.486424,4.316001,4.436970,1.498470,1.539997,0.353374,0.321598
2,398,252.410342,641.965684,604.320324,0.063052,0.051575,0.083113,0.076834,0.447264,0.465415,...,4.173962,4.718189,1.629146,2.401178,4.425915,5.277551,1.237723,1.587169,0.268964,0.222488
3,419,402.533442,604.978032,490.709738,0.043962,0.041684,0.089264,0.083791,0.391540,0.388656,...,9.709334,8.910621,2.647267,2.810882,5.631827,6.304696,2.004819,1.836838,0.414907,0.470546
4,627,409.460047,579.029937,509.311974,0.042226,0.036072,0.073829,0.081725,0.339043,0.358513,...,9.136128,9.851541,3.001734,2.822341,6.769050,6.473654,2.135641,2.037486,0.418933,0.439689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,126,354.368717,603.612277,538.287176,0.039361,0.039335,0.060274,0.057841,0.431451,0.417681,...,8.160665,9.731094,2.176555,2.282556,4.437240,4.504228,1.635577,1.413291,0.353791,0.299222
353,278,202.368505,726.632802,570.416514,0.075745,0.044148,0.074403,0.063508,0.455139,0.451087,...,4.303661,4.670829,1.475882,1.404434,3.198998,3.017578,1.199771,1.163728,0.072432,0.095027
354,142,347.532891,619.916238,531.700721,0.042164,0.036127,0.068018,0.056144,0.375827,0.397191,...,8.357738,8.542951,2.464557,2.947965,5.786969,6.389739,1.739823,1.873484,0.292447,0.436121
355,486,376.885243,557.122676,564.244986,0.077834,0.041442,0.081778,0.092392,0.466365,0.467192,...,9.664075,10.025763,2.232142,2.967895,5.573102,5.070158,1.794367,1.665238,0.270916,0.268293


In [89]:
ROI_train = ROI_train.loc[:, 'l3thVen_GM_Vol':]
ROI_test = ROI_test.loc[:, 'l3thVen_GM_Vol':]

In [90]:
GM_features = [x for x in ROI_train.columns if 'GM' in x ]
WM_features = [x for x in ROI_train.columns if 'WM' in x ]
CSF_features = [x for x in ROI_train.columns if 'CSF' in x ]

# Régression Linéaire

In [70]:
ROI_train = sm.add_constant(ROI_train)

model_stats = sm.OLS(y_train, ROI_train)

results = model_stats.fit()

In [71]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.973
Model:                            OLS   Adj. R-squared:                  0.872
Method:                 Least Squares   F-statistic:                     9.650
Date:                Sat, 03 Feb 2024   Prob (F-statistic):           6.65e-23
Time:                        20:13:08   Log-Likelihood:                -855.62
No. Observations:                 357   AIC:                             2273.
Df Residuals:                      76   BIC:                             3363.
Df Model:                         280                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   157.90

In [72]:
significant_variables = results.pvalues[results.pvalues < 0.05]

print("Variables significatives à 5% :", significant_variables)

Variables significatives à 5% : const                   0.000015
rVenVen_GM_Vol          0.024742
rOC_GM_Vol              0.012658
rAntCinGy_GM_Vol        0.044161
lFroPo_GM_Vol           0.049482
rRecGy_GM_Vol           0.046610
rSupMedFroGy_GM_Vol     0.042101
rPCu_GM_Vol             0.045959
lCbr+Mot_GM_Vol         0.046109
rTemTraGy_GM_Vol        0.037940
rCau_CSF_Vol            0.018128
lAntIns_CSF_Vol         0.019902
rAntIns_CSF_Vol         0.043623
rMedOrbGy_CSF_Vol       0.027753
lPCu_CSF_Vol            0.031325
lPosIns_CSF_Vol         0.024001
lPla_CSF_Vol            0.045399
lSupFroGy_CSF_Vol       0.004199
rSupFroGy_CSF_Vol       0.001747
lInfFroAngGy_CSF_Vol    0.028692
dtype: float64


### On remarque qu'on obtient que des features liées à la matière grise ou au liquide CSF. 

# Boosting

In [144]:
model = XGBRegressor(n_estimators=1000, random_state=42)
model_GM = XGBRegressor(n_estimators=1000, random_state=42)
model_GM_CSF = XGBRegressor(n_estimators=1000, random_state=42)

In [145]:
model.fit(ROI_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [146]:
y_pred = model.predict(ROI_test)

In [147]:
root_mean_squared_error(y_test, y_pred)

8.823513875526318

In [148]:
model_GM.fit(ROI_train[GM_features], y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [149]:
y_pred_GM = model_GM.predict(ROI_test[GM_features])

In [150]:
root_mean_squared_error(y_test, y_pred_GM)

8.47868384240385

# Optimisation XGBOOST

In [192]:
from sklearn.model_selection import GridSearchCV

xgbmodel = XGBRegressor(eval_metric = "rmse")
param_grid = {
    'n_estimators': [500, 1000, 750],
    'learning_rate': [0.001, 0.01, 0.003],
    'max_depth': [5, 10, 15],
}

# Configurer la recherche par grille avec validation croisée
grid_search = GridSearchCV(estimator=xgbmodel, param_grid=param_grid, cv=3, n_jobs=-1, verbose=1)

# Lancer la recherche£
grid_search.fit(ROI_train[GM_features], y_train)

# Afficher les meilleurs paramètres et le score correspondant
print("Meilleurs paramètres: ", grid_search.best_params_)
print("Meilleur score: ", grid_search.best_score_)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.


Meilleurs paramètres:  {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 1000}
Meilleur score:  0.6646653805375388


In [221]:
model = GradientBoostingRegressor(learning_rate = 0.01, max_depth= 3, n_estimators= 1000)

In [222]:
model.fit(ROI_train[GM_features], y_train)

GradientBoostingRegressor(learning_rate=0.01, n_estimators=1000)

In [223]:
y_pred = model.predict(ROI_test[GM_features])

In [224]:
root_mean_squared_error(y_pred, y_test)

7.66834781099084

In [227]:
len(GM_features)

142

In [226]:
ROI

,participant_id,CSF_Vol,GM_Vol,WM_Vol,l3thVen_GM_Vol,r3thVen_GM_Vol,l4thVen_GM_Vol,r4thVen_GM_Vol,lAcc_GM_Vol,rAcc_GM_Vol,...,lSupParLo_CSF_Vol,rSupParLo_CSF_Vol,lSupTemGy_CSF_Vol,rSupTemGy_CSF_Vol,lTemPo_CSF_Vol,rTemPo_CSF_Vol,lInfFroAngGy_CSF_Vol,rInfFroAngGy_CSF_Vol,lTemTraGy_CSF_Vol,rTemTraGy_CSF_Vol
0,651,334.031002,603.083565,560.465707,0.046239,0.052240,0.063827,0.068097,0.419533,0.427702,...,9.417022,8.434683,2.347495,2.171560,5.581092,5.706735,1.890379,1.612733,0.282384,0.266220
1,431,300.991328,603.101354,594.425495,0.052599,0.047901,0.088186,0.105241,0.373451,0.377817,...,7.536400,9.255206,1.916260,2.486424,4.316001,4.436970,1.498470,1.539997,0.353374,0.321598
2,398,252.410342,641.965684,604.320324,0.063052,0.051575,0.083113,0.076834,0.447264,0.465415,...,4.173962,4.718189,1.629146,2.401178,4.425915,5.277551,1.237723,1.587169,0.268964,0.222488
3,419,402.533442,604.978032,490.709738,0.043962,0.041684,0.089264,0.083791,0.391540,0.388656,...,9.709334,8.910621,2.647267,2.810882,5.631827,6.304696,2.004819,1.836838,0.414907,0.470546
4,627,409.460047,579.029937,509.311974,0.042226,0.036072,0.073829,0.081725,0.339043,0.358513,...,9.136128,9.851541,3.001734,2.822341,6.769050,6.473654,2.135641,2.037486,0.418933,0.439689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,126,354.368717,603.612277,538.287176,0.039361,0.039335,0.060274,0.057841,0.431451,0.417681,...,8.160665,9.731094,2.176555,2.282556,4.437240,4.504228,1.635577,1.413291,0.353791,0.299222
353,278,202.368505,726.632802,570.416514,0.075745,0.044148,0.074403,0.063508,0.455139,0.451087,...,4.303661,4.670829,1.475882,1.404434,3.198998,3.017578,1.199771,1.163728,0.072432,0.095027
354,142,347.532891,619.916238,531.700721,0.042164,0.036127,0.068018,0.056144,0.375827,0.397191,...,8.357738,8.542951,2.464557,2.947965,5.786969,6.389739,1.739823,1.873484,0.292447,0.436121
355,486,376.885243,557.122676,564.244986,0.077834,0.041442,0.081778,0.092392,0.466365,0.467192,...,9.664075,10.025763,2.232142,2.967895,5.573102,5.070158,1.794367,1.665238,0.270916,0.268293


In [230]:
test = ROI_train.copy()

In [231]:
test["age"] = y_train

In [232]:
test.corr()["age"]

participant_id          0.341417
CSF_Vol                 0.802981
GM_Vol                 -0.820098
WM_Vol                 -0.354918
l3thVen_GM_Vol         -0.482567
                          ...   
lInfFroAngGy_CSF_Vol    0.629072
rInfFroAngGy_CSF_Vol    0.615305
lTemTraGy_CSF_Vol       0.653696
rTemTraGy_CSF_Vol       0.654878
age                     1.000000
Name: age, Length: 289, dtype: float64

In [240]:
ROI.columns.to_list()[250:]

['lPCu_CSF_Vol',
 'rPCu_CSF_Vol',
 'lParHipGy_CSF_Vol',
 'rParHipGy_CSF_Vol',
 'lPosIns_CSF_Vol',
 'rPosIns_CSF_Vol',
 'lParOpe_CSF_Vol',
 'rParOpe_CSF_Vol',
 'lPoCGy_CSF_Vol',
 'rPoCGy_CSF_Vol',
 'lPosOrbGy_CSF_Vol',
 'rPosOrbGy_CSF_Vol',
 'lPla_CSF_Vol',
 'rPla_CSF_Vol',
 'lPrcGy_CSF_Vol',
 'rPrcGy_CSF_Vol',
 'lTem_CSF_Vol',
 'rTem_CSF_Vol',
 'lSCA_CSF_Vol',
 'rSCA_CSF_Vol',
 'lSupFroGy_CSF_Vol',
 'rSupFroGy_CSF_Vol',
 'lCbr+Mot_CSF_Vol',
 'rCbr+Mot_CSF_Vol',
 'lSupMarGy_CSF_Vol',
 'rSupMarGy_CSF_Vol',
 'lSupOccGy_CSF_Vol',
 'rSupOccGy_CSF_Vol',
 'lSupParLo_CSF_Vol',
 'rSupParLo_CSF_Vol',
 'lSupTemGy_CSF_Vol',
 'rSupTemGy_CSF_Vol',
 'lTemPo_CSF_Vol',
 'rTemPo_CSF_Vol',
 'lInfFroAngGy_CSF_Vol',
 'rInfFroAngGy_CSF_Vol',
 'lTemTraGy_CSF_Vol',
 'rTemTraGy_CSF_Vol']